In [27]:
import csv
import re
import sys
from lxml import etree
from tqdm import tqdm

In [11]:
inputFile = '../data/source/SFF-Datenbank-Export.csv'

In [12]:
artistsFile = '../data/source/sff-curation-artists.csv'
curatedFilesPre = '../data/source/sff-curation'
outputDirectory = '../data/xml/sff/'
outputPrefix = 'sff-record-'

In [13]:
curatedFields = ['Keywords', 'Ortsbezug']

In [39]:
limit = 10
offset = 20
idsToOutput = False

In [40]:
inputData = []
with open(inputFile, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        inputData.append(row)

In [61]:
def cleanKeyForTags(key):
    cleanedKey = re.sub(r'[\s.*_]', '', key)
    cleanedKey = re.sub(r'[()]', '-', cleanedKey)
    cleanedKey = re.sub(r'-$', '', cleanedKey)
    print(key,cleanedKey)
    return cleanedKey

def convertRowToXMLRecord(row):
    record = etree.Element('record')
    for k in row.keys():
        if k:
            tag = cleanKeyForTags(k)
            subElement = etree.SubElement(record, tag)
            subElement.text = row[k]
    return record

def splitMultiValueFields(record):
    multiValueSeparators = {
        "KünsterIn": "/",
        "Bemerkungen": "/",
        "Keywords": ",",
        "Ortsbezug": r"\)[,|;]"
    }
    # Add suffix that may be cut off through regex separator
    multiValueSuffixes = {
        "Ortsbezug": ")"
    }
    for key in multiValueSeparators.keys():
        tag = record.find(cleanKeyForTags(key))
        values = re.split(multiValueSeparators[key], tag.text)
        values = [d.strip() for d in values]
        if key in multiValueSuffixes.keys():
            values = [d + multiValueSuffixes[key] for d in values[:-1]] + values[-1:]
        
        valuesTag = etree.SubElement(tag, 'values')
        for value in values:
            valueTag = etree.SubElement(valuesTag, 'value')
            etree.SubElement(valueTag, 'text').text = value

        tag.text = ''
        
    return record 

collection = etree.XML("<collection/>")

for row in tqdm(inputData[offset:offset + limit]):
    record = convertRowToXMLRecord(row)
    record = splitMultiValueFields(record)
    
    collection.clear()
    collection.append(record)
    outputFile = outputDirectory + outputPrefix + record.find("InvNr").text + ".xml"
    with open(outputFile, 'wb') as f:
        f.write(etree.tostring(collection, xml_declaration=True, encoding='UTF-8', pretty_print=True))
        f.close()

100%|██████████| 10/10 [00:00<00:00, 54.18it/s]

Inv Nr. Intern InvNrIntern
KünsterIn KünsterIn
Titel Titel
Entst. von Entstvon
Entst. bis Entstbis
Datierung Datierung
Technik Technik
Zuständigkeit Zuständigkeit
Werkstatus Werkstatus
Kaufm. Status KaufmStatus
Salestatus Salestatus
Edition Edition
Material Material
Material_2 Material2
Edition Übers. EditionÜbers
Bemerkungen Bemerkungen
Bez. Künstler BezKünstler
Bez. fremd Bezfremd
Zertifikat Zertifikat
Zertifikat Übers. ZertifikatÜbers
Titel Übers. TitelÜbers
Technik Übers. TechnikÜbers
Inv. Datum InvDatum
Provenienz Provenienz
Gattung Gattung
Titel Übers. 2 TitelÜbers2
Inv. Nr. InvNr
Keywords Keywords
Technik Übers. 2 TechnikÜbers2
Edition Übers. 2 EditionÜbers2
Ortsbezug Ortsbezug
Masse1 Suffix Masse1Suffix
Masse1 Prefix Masse1Prefix
Ref. Nr. RefNr
Inschrift Übers. 2 InschriftÜbers2
Zert. Übers. 2 ZertÜbers2
Datumstempel Datumstempel
Zeitstempel Zeitstempel
Ueberprüft am Ueberprüftam
Ueberprüft von Ueberprüftvon
Sortation Sortation
pID pID
Ort (Standort) Ort-Standort
Bereich (Stand